In [1]:
import pandas as pd
df_e = pd.read_csv('e.csv')
df_e.info()
df_e.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   wiki_id  837 non-null    object
dtypes: object(1)
memory usage: 6.7+ KB


,wiki_id
0,Q43513
1,Q43436
2,Q5283
3,Q138979
4,Q43088


In [2]:
entity_list = list(df_e['wiki_id'])
print(len(entity_list))
entity_list[:3]

837


['Q43513', 'Q43436', 'Q5283']

In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON
def query_subject(entity_id='Q43088'):
    # SPARQL endpoint for Wikidata
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = f"""
    SELECT ?subjectLabel ?predicateLabel ?objectLabel ?subject ?object 
    WHERE {{
      ?subject ?predicate ?object.
      ?subject rdfs:label ?subjectLabel.
      ?x wikibase:directClaim ?predicate.
      ?x rdfs:label ?predicateLabel.
      ?object rdfs:label ?objectLabel.
      BIND(wd:{entity_id} AS ?subject) 
      FILTER(LANG(?subjectLabel) = "en").
      FILTER(LANG(?predicateLabel) = "en").
      FILTER(LANG(?objectLabel) = "en").
    }}
    LIMIT 10000
    """
    # Set the query and return format
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    # Execute the query
    results = sparql.query().convert()
    return results

# query()

In [16]:
def query_object(entity_id='Q43088'):
    # SPARQL endpoint for Wikidata
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = f"""
    SELECT ?subjectLabel ?predicateLabel ?objectLabel ?subject ?object 
    WHERE {{
      ?subject ?predicate ?object.
      ?subject rdfs:label ?subjectLabel.
      ?x wikibase:directClaim ?predicate.
      ?x rdfs:label ?predicateLabel.
      ?object rdfs:label ?objectLabel.
      BIND(wd:{entity_id} AS ?object) 
      FILTER(LANG(?subjectLabel) = "en").
      FILTER(LANG(?predicateLabel) = "en").
      FILTER(LANG(?objectLabel) = "en").
    }}
    LIMIT 10000
    """
    # Set the query and return format
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    # Execute the query
    results = sparql.query().convert()
    return results
    
# query_object('Q43088')

In [17]:
import pandas as pd
def e2spo(entity_id='Q43088'):
    csv_data = []  # CSV header
    
    results = query_subject(entity_id)    
    for result in results["results"]["bindings"]:
        subject_label = result["subjectLabel"]["value"]
        predicate_label = result["predicateLabel"]["value"]
        object_label = result["objectLabel"]["value"]
        
        # Extract Wikidata IDs
        subject_id = result["subject"]["value"].split('/')[-1]  # Extract subject Wikidata ID
        object_id = result["object"]["value"].split('/')[-1]    # Extract object Wikidata ID
        
        # Add SPO triples to CSV data
        csv_data.append([subject_label, predicate_label, object_label, subject_id, object_id])

    results = query_object(entity_id)    
    for result in results["results"]["bindings"]:
        subject_label = result["subjectLabel"]["value"]
        predicate_label = result["predicateLabel"]["value"]
        object_label = result["objectLabel"]["value"]
        
        # Extract Wikidata IDs
        subject_id = result["subject"]["value"].split('/')[-1]  # Extract subject Wikidata ID
        object_id = result["object"]["value"].split('/')[-1]    # Extract object Wikidata ID
        
        # Add SPO triples to CSV data
        csv_data.append([subject_label, predicate_label, object_label, subject_id, object_id])
    
    df = pd.DataFrame(
        csv_data,
        columns=["subjectLabel","predicateLabel","objectLabel","subject","object"]
    )
    return df
    
df = e2spo(entity_id='Q43088')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    174 non-null    object
 1   predicateLabel  174 non-null    object
 2   objectLabel     174 non-null    object
 3   subject         174 non-null    object
 4   object          174 non-null    object
dtypes: object(5)
memory usage: 6.9+ KB


,subjectLabel,predicateLabel,objectLabel,subject,object
0,ruby,instance of,mineral variety,Q43088,Q429795
1,ruby,described by source,Brockhaus and Efron Encyclopedic Dictionary,Q43088,Q602358
2,ruby,described by source,Encyclopædia Britannica 11th edition,Q43088,Q867541
3,ruby,described by source,Explanatory Dictionary of the Living Great Rus...,Q43088,Q1970746
4,ruby,described by source,The Nuttall Encyclopædia,Q43088,Q3181656


In [18]:
df.tail()

,subjectLabel,predicateLabel,objectLabel,subject,object
169,Steine und Erden,main subject,ruby,Q19229388,Q43088
170,1911 Encyclopædia Britannica/Ruby,main subject,ruby,Q84683979,Q43088
171,ruby maser,uses,ruby,Q96217685,Q43088
172,Ruby,inspired by,ruby,Q26714785,Q43088
173,Category:Ruby,category's main topic,ruby,Q24272721,Q43088


In [19]:
from tqdm.auto import tqdm
list_df=[]
pbar = tqdm(entity_list)
for e in pbar:
    pbar.set_description(f'Query entity: {e:<15}')
    df=e2spo(entity_id = e)
    df['source']=e
    list_df.append(df)

# Concatenate the DataFrames row-wise (axis=0)
df = pd.concat(list_df, axis=0, ignore_index=True)
df.info()
df.head()

  0%|          | 0/837 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754311 entries, 0 to 754310
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   subjectLabel    754311 non-null  object
 1   predicateLabel  754311 non-null  object
 2   objectLabel     754311 non-null  object
 3   subject         754311 non-null  object
 4   object          754311 non-null  object
 5   source          754311 non-null  object
dtypes: object(6)
memory usage: 34.5+ MB


,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,emerald,instance of,mineral variety,Q43513,Q429795,Q43513
1,emerald,described by source,Bible Encyclopedia of Archimandrite Nicephorus,Q43513,Q4086271,Q43513
2,emerald,described by source,The New Student's Reference Work,Q43513,Q16082057,Q43513
3,emerald,described by source,Small Brockhaus and Efron Encyclopedic Dictionary,Q43513,Q19180675,Q43513
4,emerald,described by source,Explanatory Dictionary of the Living Great Rus...,Q43513,Q23705356,Q43513


In [20]:
df_pivot = pd.pivot_table(
    df.reset_index(),
    # columns=['col1'],
    index=['predicateLabel'],
    values=['index'], 
    aggfunc={
        'index': ["count"],
    },
    fill_value=0
)
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]
df_pivot.sort_values('index_count',ascending=False).head(50)

,index_count
predicateLabel,
country,142480
occupation,101010
country of citizenship,84650
instance of,74133
main subject,64500
made from material,37728
located in the administrative territorial entity,24164
color,18241
field of work,15637


In [21]:
values_to_drop = {
    'diplomatic relation',
    'contains the administrative territorial entity',
    'emergency phone number',
    'topic\'s main Wikimedia portal',
    'public holiday',
    'highest judicial authority',
    'Wikidata property',
    'Wikimedia outline',
    'described by source',
    'different from',
    "topic's main category",
    'on focus list of Wikimedia project',
    "topic's main template",
    'maintained by WikiProject',
    'located in the administrative territorial entity',
    'top-level Internet domain',
    'flag',
    'currency',
    'language used'
}
df2 = df[~df['predicateLabel'].isin(values_to_drop)]
df_pivot = pd.pivot_table(
    df2.reset_index(),
    # columns=['col1'],
    index=['predicateLabel'],
    values=['index'], 
    aggfunc={
        'index': ["count"],
    },
    fill_value=0
)
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]
df_pivot.sort_values('index_count',ascending=False).head(50)

,index_count
predicateLabel,
country,142480
occupation,101010
country of citizenship,84650
instance of,74133
main subject,64500
made from material,37728
color,18241
field of work,15637
country of origin,15207


In [22]:
df2.to_csv('spo_e.csv')
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 720255 entries, 0 to 754310
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   subjectLabel    720255 non-null  object
 1   predicateLabel  720255 non-null  object
 2   objectLabel     720255 non-null  object
 3   subject         720255 non-null  object
 4   object          720255 non-null  object
 5   source          720255 non-null  object
dtypes: object(6)
memory usage: 38.5+ MB


,subjectLabel,predicateLabel,objectLabel,subject,object,source
0,emerald,instance of,mineral variety,Q43513,Q429795,Q43513
7,emerald,subclass of,gemstone,Q43513,Q83437,Q43513
8,emerald,subclass of,beryl,Q43513,Q103480,Q43513
9,emerald,streak color,white,Q43513,Q23444,Q43513
10,emerald,crystal system,hexagonal crystal system,Q43513,Q663314,Q43513


In [23]:
import json
def spo2kg(df_spo_wiki):
    df = df_spo_wiki
    df_spo = df_spo_wiki[['subjectLabel','predicateLabel','objectLabel']]
    
    metadata = {}
    # Base Wikipedia URL for metadata
    wiki_base_url = "https://en.wikipedia.org/wiki/"
    
    for i in df.itertuples():
        # print(i.subjectLabel)
        
        # Add subject metadata (Wikidata ID and Wikipedia URL)
        if i.subjectLabel not in metadata:
            metadata[i.subjectLabel] = {
                "url": f"{wiki_base_url}{i.subjectLabel.replace(' ', '_')}",
                "wiki_id": i.subject
            }
        
        # Add object metadata (Wikidata ID and Wikipedia URL)
        if i.objectLabel not in metadata:
            metadata[i.objectLabel] = {
                "url": f"{wiki_base_url}{i.objectLabel.replace(' ', '_')}",
                "wiki_id": i.object
            }
    
    return df_spo, metadata

df_spo, metadata = spo2kg(df2)
df_spo.info()
df_spo.head()

<class 'pandas.core.frame.DataFrame'>
Index: 720255 entries, 0 to 754310
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   subjectLabel    720255 non-null  object
 1   predicateLabel  720255 non-null  object
 2   objectLabel     720255 non-null  object
dtypes: object(3)
memory usage: 22.0+ MB


,subjectLabel,predicateLabel,objectLabel
0,emerald,instance of,mineral variety
7,emerald,subclass of,gemstone
8,emerald,subclass of,beryl
9,emerald,streak color,white
10,emerald,crystal system,hexagonal crystal system


In [24]:
csv_file= 'data_e.csv'
metadata_file='metadata_e.json'

df_spo.to_csv(csv_file, header=False, index=False)
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2)

In [25]:
!kg add -f data_e.csv
!kg meta -f metadata_e.json
!kg start

🎉 Starting the app.
 * Serving Flask app 'kgsearch.app.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Oct/2024 19:56:28] "GET /search/1/1/1/blue;gemstone HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:56:46] "GET /search/1/1/1/blue;gemstone; HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 19:57:04] "GET /search/1/1/2/blue;gemstone; HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 20:02:42] "GET /search/1/1/2/blue;gemstone;mi HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 20:02:46] "GET /search/1/1/2/blue;gemstone;minera HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 20:02:49] "GET /search/1/1/2/blue;gemstone;mineral HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 20:03:05] "GET /search/1/1/2/blue;gemstone;mineral; HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 20:03:17] "GET /search/1/1/2/blue;gemstone;mineral;mineral%20s HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2024 20:03:35] "GET /s